In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from utils import to_timeseries_input, splitDate, Trainer, PredictReconstruction, PlotResult
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader

### Introduction to Recurrent Neural Networks

In this TP, we will introduce Recurrent Neural Networks (RNNs) and demonstrate how to use them to model time-series data. Specifically, we will be working with the CUBEMS datasets, which contains measurements of temperature, humidity, and light intensity in a 7 floors buildings in Bangkok, Thailand. We will use only a small portion of the data consisting of measurement from 1 zones on the 7th floors. You can explore the dataset by running the cell below.

In [3]:
path = "data.csv"
data = pd.read_csv(path)
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

For this TP, we will focus on predicting the humidity measurement. We will split the dataset to train and test by taking a date and limit the test data to only 1 months for visualization. 

In [4]:
# Select the columns to use
col = ["humidity"] # 
data = data[[*col]]

date_limit = pd.to_datetime("2019-09-20")
date_limit2 = pd.to_datetime("2019-09-30")
train_date, test_date = splitDate(data.index, date_limit)
train_data = data.loc[data.index < date_limit][col].values
test_data = data.loc[(data.index >= date_limit) & (data.index < date_limit2)][col].values

scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
# To time series input
lookback = 24  
lookahead = 1 
batch_size = 128

# Train and validation data
train_input, train_output = to_timeseries_input(train_data, lookback, lookahead) 
trainx, validx, trainy, validy = train_test_split(train_input, train_output, test_size=0.3, shuffle=False) 

train_tensor = TensorDataset(torch.from_numpy(trainx).float(), torch.from_numpy(trainy).float()) 
val_tensor = TensorDataset(torch.from_numpy(validx).float(), torch.from_numpy(validy).float()) 

train_loader = DataLoader(dataset=train_tensor, batch_size=batch_size, shuffle=False) 
val_loader = DataLoader(dataset=val_tensor, batch_size=batch_size, shuffle=False)  


# Test data
test_input, test_output = to_timeseries_input(test_data, lookback, lookahead) 
test_tensor = TensorDataset(torch.from_numpy(test_input).float(), torch.from_numpy(test_output).float()) 
test_loader = DataLoader(dataset=test_tensor, batch_size=batch_size, shuffle=False) 

## Recurrent Neural Network
To build a simple Recurrent Neural Network (RNN) model, we will start with an input layer that takes a sequence of humidity measurements as input. The input tensor has a shape of `(batch_size, T, 1)`, where `batch_size` is the number of sequences in the batch, `T` is the length of each sequence, and `1` is the size of the input vector at each time step.

Next, we will add a hidden layer that is an RNN layer with `hidden_size` units. The hidden layer takes the input sequence and maintains a hidden state vector of size `hidden_size` at each time step. The output of the hidden layer has shape `(batch_size, T, hidden_size)`.

Finally, we will add an output layer that is a fully connected layer with 1 unit. The output layer takes the output vector from the hidden layer and computes the predicted value for `1` timesteps. The output tensor has shape `(batch_size, 1, 1)`.

#### Your job: 
Complete the code below.

In [ ]:
hidden_size = 16 
in_features = 1 
out_features = 1 
output_length = lookahead
epochs = 20
patience = 500

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, in_features, hidden_size, out_features, out_len):
        """
        Initialize the SimpleRNN model.
        Args:
            in_features (int): The number of input features.
            hidden_size (int): The number of features in the hidden state.
            out_features (int): The number of output features.
            out_len (int): The length of the output sequence.
        """
        super(SimpleRNN,self).__init__()
        self.out_len = out_len
        self.rnn = nn.RNN(in_features, hidden_size, batch_first=True, dropout=0.3) 
        self.fc = nn.Linear(hidden_size, out_features) 
    def forward(self,x):
        out, h_en = self.rnn(x) 
        out = self.fc(out[:, -self.out_len:, :]) 
        return out
    

In [ ]:
model = SimpleRNN(in_features, hidden_size, out_features, output_length).to(device) 
optimizer = torch.optim.Adam(model.parameters()) 
criterion = nn.MSELoss() 
# Train Loop
trainer = Trainer(model, train_loader=train_loader,  
                  val_loader=val_loader, 
                  test_loader=test_loader, 
                  criterion = criterion, 
                  optimizer = optimizer, 
                  device = device) 

In [ ]:
train_loss, train_metric, val_loss, val_metrics = trainer.train(epochs=epochs, patience=patience)

In [ ]:
trainer.plot_metrics(train_loss, val_loss, train_metric, val_metrics)

In [ ]:
target_rnn, pred_rnn = PredictReconstruction(model, test_loader, lookahead,device)
PlotResult(target_rnn, pred_rnn)

## Exploring Long Short Term Memory (LSTM)

In this section, we are going to explore another type of RNN model, namely Long Short Term Memory (LSTM). Unlike traditional Recurrent Neural Networks (RNNs), which struggle to maintain and utilize information from earlier steps in long sequences due to the vanishing gradient problem, LSTMs are designed to effectively capture long-term dependencies in sequence data.

**Your task**: 

Complete the model definition.

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, in_features, hidden_size , out_features, out_len):
        super(SimpleLSTM, self).__init__()
        self.out_len = out_len
        self.lstm = nn.LSTM(in_features, hidden_size, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, out_features)
        
    def forward(self, x):
        #TODO Complet the forward pass
        out, (h_en,c_en) = self.lstm(x)
        out = self.fc(out[:,-self.out_len:,:]) 
        return out

In [ ]:
lstm_model = SimpleLSTM(in_features, hidden_size, out_features, output_length).to(device)
optimizer = torch.optim.Adam(lstm_model.parameters())
criterion = nn.MSELoss()
trainer = Trainer(lstm_model, train_loader=train_loader, 
                  val_loader=val_loader, 
                  test_loader=test_loader,
                  criterion = criterion, 
                  optimizer = optimizer,
                  device = device)

In [ ]:
train_loss, train_metric, val_loss, val_metrics = trainer.train(epochs=epochs, patience=patience)

In [ ]:
trainer.plot_metrics(train_loss, val_loss, train_metric, val_metrics)

In [ ]:
target_lstm, pred_lstm = PredictReconstruction(lstm_model, test_loader, lookahead,device)
PlotResult(target_lstm, pred_lstm)

## Many-to-Many RNN Architecture

The existing code provides a one-step prediction for each time-series. We will now modify this to explore a many-to-many RNN architecture.

Your task is to adapt the existing cells for many-to-many prediction. Here are some suggestions to guide you:

1. **Data Splitting**: Begin by adjusting the data split. Experiment with setting the `lookback` equal to `lookahead`, and then try making `lookback` greater than `lookahead`.

2. **Modify the RNN and LSTM Models**: Adjust the SimpleRNN and Simple LSTM models to accommodate the many-to-many setting.

3. **Training**: Train the model with the new many-to-many setting.

After completing these steps, observe the prediction quality. How does the many-to-many model compare to the original many-to-one model in terms of prediction quality?